In [1]:
%pwd

'/Users/ryandevera/data-science/umn_environments/Deeplifting/deeplifting/notebooks'

In [2]:
%cd ../..

/Users/ryandevera/data-science/umn_environments/Deeplifting


In [3]:
%ls

LJ_data/                        high-dimension-paper-results/
LJ_data.tar                     high-dimension-search-results/
README.md                       images/
Untitled.ipynb                  jobs/
__pycache__/                    low-dimension-search-results/
algorithm_compare_results/      models/
config.py                       paper-images/
data/                           requirements.txt
data-queue/                     results/
data-queue-2023-09-24/          search_results/
data-queue-2023-10-18/          tasks.py
deeplifting/                    test-low-dimension-results.png
deeplifting.png                 wandb/
experiments/


In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
# stdlib
import click
import os
import warnings
from datetime import datetime
from itertools import product

# third party
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
from pygranso.private.getNvar import getNvarTorch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct

# first party
from config import (
    low_dimensional_problem_names,
    search_agg_functions,
    search_hidden_activations,
    search_hidden_sizes,
    search_include_bn,
    search_input_sizes,
    search_output_activations,
)
from deeplifting.models import ReLUDeepliftingMLP
from deeplifting.optimization import deeplifting_predictions, deeplifting_nd_fn
from deeplifting.problems import HIGH_DIMENSIONAL_PROBLEMS_BY_NAME, PROBLEMS_BY_NAME
from deeplifting.utils import (
    get_devices,
    initialize_vector,
    set_seed,
    train_model_to_output,
)

# Filter warnings
warnings.filterwarnings('ignore')

In [ ]:
torch.cuda.is_available()

In [ ]:
!nvidia-smi

In [ ]:
from tasks import find_best_architecture_task

In [ ]:
problem_name = 'ackley'
method = 'deeplifting-pygranso'
dimensionality = 'low-dimensional'
experimentation = True

In [ ]:
find_best_architecture_task(
    problem_name=problem_name,
    method=method,
    dimensionality=dimensionality,
    experimentation=False,
    include_weight_initialization=True,
)

In [ ]:
%debug